# TFX - Run Training Pipeline locally using BeamDagRunner

The purpose of this notebook is the to test running the end-to-end TFX pipeline locally. The TFX pipeline is implemented in the [tfx_pipeline](tfx_pipeline) directory. The notebook covers the following tasks:
1. Set the pipeline configurations
2. Configure the local metastore
3. Run the pipeline 

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import kfp
import tfx
from tfx.orchestration.local.local_dag_runner import LocalDagRunner
import tensorflow as tf
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2
import logging

logging.getLogger().setLevel(logging.INFO)

print("Tensorflow Version:", tfx.__version__)
print("KFP Version:", kfp.__version__)

## 1. Set pipeline configurations

In [ ]:
os.environ["DATASET_DISPLAY_NAME"] = 'chicago_taxi_tips'
os.environ["PROJECT"] = 'ksalama-cloudml'
os.environ["REGION"] = 'us-central1'
os.environ["GCS_LOCATION"] = "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/pipelines_local_runner"
os.environ["TRAIN_LIMIT"] = "85000"
os.environ["TEST_LIMIT"] = "15000"
os.environ["BEAM_RUNNER"] = "DirectRunner"
os.environ["TRAINING_RUNNER"] = "local"

In [ ]:
from src.pipelines import config
for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

## 2. Congifure local metadata store

In [ ]:
MLMD_SQLLITE = 'mlmd.sqllite'

gcs_location = os.environ["GCS_LOCATION"]
print(gcs_location)

if tf.io.gfile.exists(gcs_location):
    print("Removing previous artifacts...")
    tf.io.gfile.rmtree(gcs_location)

if tf.io.gfile.exists(MLMD_SQLLITE):
    print("Removing local mlmd SQLite...")
    tf.io.gfile.remove(MLMD_SQLLITE)

metadata_connection_config = metadata_store_pb2.ConnectionConfig()
metadata_connection_config.sqlite.filename_uri = MLMD_SQLLITE
metadata_connection_config.sqlite.connection_mode = 3
print("ML metadata store is ready.")

## 3. Run the pipeline

In [ ]:
from src.pipelines import training_pipeline

In [ ]:
pipeline_root = os.path.join(
    config.ARTIFACT_STORE_URI,
    config.PIPELINE_NAME,
)

runner = LocalDagRunner()

pipeline = training_pipeline.create_pipeline(
    metadata_connection_config=metadata_connection_config,
    pipeline_root=pipeline_root,
    num_epochs=50,
    batch_size=512,
    learning_rate=0.0003,
    hidden_units="256,128",
)

runner.run(pipeline)

print("Pipeline finished exection.")